# How to use XLEMOO: Explainable Learnable Multiobjective Optimization

XLEMOO is a hybrid evolutionary-ML approach that combines evolutionary algorithms
(Darwinian mode) with interpretable machine learning (Learning mode) to find
near-Pareto optimal solutions while providing **explainability** about what makes
solutions good.

The core idea is that by training interpretable ML models on the population history,
we can extract human-readable rules describing what characterizes high-performing
solutions. These rules can then be used to instantiate new candidate solutions
directly.

**Reference:**
Misitano, G. (2024). Towards Explainable Multiobjective Optimization: XLEMOO.
*ACM Trans. Evol. Learn. Optim.*, 4(1). https://doi.org/10.1145/3626104

**Prerequisites:** Before proceeding, make sure you are familiar with:

- [How evolutionary algorithms are structured in DESDEO](../../explanation/templates_and_pub_sub)
- [How to configure evolutionary algorithms with the Pydantic interface](../ea_options)
- [How multiobjective optimization problems are defined](../../explanation/problem_format.ipynb)

## How XLEMOO works

XLEMOO alternates between two modes:

**Darwinian mode** uses a standard evolutionary algorithm with crossover,
mutation, and selection operators. The selection is done by a
`ScalarizationSelector` that ranks solutions by a **scalarization column**
(e.g., weighted sums or ASF) computed by the shared evaluator.

**Learning mode** replaces the standard evolutionary operators with an ML-based
pipeline:

1. Read population history from the archive.
2. Rank solutions by the **same scalarization column** (computed by the evaluator
   and stored in the archive) and split into a **H-group** (high-performing)
   and **L-group** (low-performing).
3. Train an interpretable ML classifier to distinguish H from L.
4. Extract rules from the classifier and instantiate new candidate
   solutions based on those rules.
5. Evaluate the candidates using the shared evaluator.
6. Select the best solutions using the same ScalarizationSelector.

The key design principle is that both modes use the **same fitness criterion**:
a scalarization function added to the Problem. The `EMOEvaluator` computes it
for every solution, and both the `ScalarizationSelector` and the
`XLEMOOInstantiator` read the same column from the outputs/archive.

```
Darwinian mode (N iterations):
  offspring       = crossover(population)
  offspring       = mutate(offspring)
  offspring_eval  = evaluator.evaluate(offspring)   # computes scalarization
  population      = selection(parents, offspring)    # ranks by scalarization

Learning mode (M iterations):
  candidates      = instantiator(population)         # ML rules -> variables
  candidates_eval = evaluator.evaluate(candidates)   # computes scalarization
  population      = selection(parents, candidates)   # same ranking
```

Because the evaluation step goes through the shared `EMOEvaluator`,
all components (terminator, archives, etc.) are correctly notified about
the function evaluations performed during learning mode.

## Running XLEMOO with ASF-guided learning

The original XLEMOO paper recommends using the **Achievement Scalarizing
Function (ASF)** as the fitness criterion for the H/L group ranking. ASF
incorporates a reference point provided by the decision maker, guiding
the learning mode to focus on solutions near that preferred region.

We use `xlemoo_options()` as the base configuration and customize the
`ScalarizationSpec` to use ASF. We demonstrate on the DTLZ2 test problem
(3 objectives, 6 variables). The true Pareto front of DTLZ2 lies on the
unit sphere, and the distance variables ($x_3$ through $x_6$) should
converge to 0.5.

The parameters below match those used in the original XLEMOO experiments:
- **SkopeRulesClassifier** as the ML model (the default in the paper)
- **h_split=0.2, l_split=0.2** (top/bottom 20% for H/L groups)
- **instantiation_factor=10** (generate 10x the population size in candidates)
- **unique_only=True** (deduplicate before training)
- **19 Darwinian + 1 Learning** iterations per cycle

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np

from desdeo.emo.options.algorithms import xlemoo_options
from desdeo.emo.options.scalarization_selection import ScalarizationSpec
from desdeo.emo.options.templates import emo_constructor
from desdeo.emo.options.xlemoo_selection import XLEMOOSelectorOptions
from desdeo.problem.testproblems import dtlz2

# Define a 3-objective, 6-variable DTLZ2 problem
problem = dtlz2(n_objectives=3, n_variables=6)

# Get pre-configured XLEMOO options
options = xlemoo_options()

# Use ASF scalarization with a centered reference point
reference_point = [0.5, 0.5, 0.5]
options.template.scalarization = ScalarizationSpec(
    type="asf",
    symbol="scal_fitness",
    reference_point=reference_point,
)

# Configure learning mode to match the original paper
options.template.learning_instantiator = XLEMOOSelectorOptions(
    ml_model_type="SkopeRules",
    ml_model_kwargs={
        "precision_min": 0.1,
        "n_estimators": 30,
        "max_depth": None,
    },
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=10.0,
    unique_only=True,
)

# 19 Darwinian + 1 Learning per cycle (matching the original paper)
options.template.darwin_iterations_per_cycle = 19
options.template.learning_iterations_per_cycle = 1

# Build and run the solver
solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")
print(results.optimal_outputs.head())

## Visualizing the evolution

Rather than showing just the final Pareto front, we can animate the
**selected population per generation** to see how the solutions evolve
over time. The learning archive stores all evaluated solutions with
generation numbers, and the base archive stores the selected population
per generation in the `selections` attribute.

In [ ]:
import plotly.graph_objects as go

# The learning archive stores selected populations per generation
sel_df = extras.learning_archive.selections

# Build animation frames — one per generation
generations = sorted(sel_df["generation"].unique().to_list())

frames = []
for gen in generations:
    gen_data = sel_df.filter(sel_df["generation"] == gen)
    frames.append(
        go.Frame(
            data=[
                go.Scatter3d(
                    x=gen_data["f_1"].to_list(),
                    y=gen_data["f_2"].to_list(),
                    z=gen_data["f_3"].to_list(),
                    mode="markers",
                    marker=dict(size=3, color="blue", opacity=0.7),
                )
            ],
            name=str(gen),
        )
    )

# Initial frame
init = sel_df.filter(sel_df["generation"] == generations[0])

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=init["f_1"].to_list(),
            y=init["f_2"].to_list(),
            z=init["f_3"].to_list(),
            mode="markers",
            marker=dict(size=3, color="blue", opacity=0.7),
        )
    ],
    frames=frames,
    layout=go.Layout(
        title="XLEMOO population evolution on DTLZ2",
        scene=dict(
            xaxis=dict(title="f_1", range=[0, 1.5]),
            yaxis=dict(title="f_2", range=[0, 1.5]),
            zaxis=dict(title="f_3", range=[0, 1.5]),
        ),
        updatemenus=[
            dict(
                type="buttons",
                showactive=False,
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            dict(
                                frame=dict(duration=200, redraw=True),
                                fromcurrent=True,
                            ),
                        ],
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[
                            [None],
                            dict(
                                frame=dict(duration=0, redraw=False),
                                mode="immediate",
                            ),
                        ],
                    ),
                ],
            )
        ],
        sliders=[
            dict(
                active=0,
                steps=[
                    dict(
                        args=[[str(gen)], dict(frame=dict(duration=0, redraw=True), mode="immediate")],
                        label=str(gen),
                        method="animate",
                    )
                    for gen in generations
                ],
                currentvalue=dict(prefix="Generation: "),
            )
        ],
    ),
)
fig.show(renderer="notebook", include_plotlyjs="cdn")

In [ ]:
# Verify proximity to unit sphere (final non-dominated front)
obj_values = results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
norms = np.sqrt(np.sum(obj_values**2, axis=1))
print(f"Median distance to unit sphere: {np.median(norms):.4f}")
print(f"Min: {norms.min():.4f}, Max: {norms.max():.4f}")

## Extracting and interpreting rules

A key feature of XLEMOO is the ability to extract **human-readable rules**
from the ML model that describe what characterizes good solutions. This is
the "explainable" part of XLEMOO.

The `XLEMOOInstantiator` (accessible via `extras.learning_instantiator`)
stores the most recently trained classifier and its extracted rules. Since
we configured the learning mode with SkopeRules, `last_rules` contains the
ruleset and precisions from the final learning cycle — no need to re-train
a separate model.

For DTLZ2, the true Pareto-optimal solutions have $x_i = 0.5$ for the
distance variables ($i \geq 3$), so we expect the rules to reflect
conditions near $x_i \approx 0.5$ for those variables.

In [ ]:
# Access the most recently trained SkopeRules model and its rules
instantiator = extras.learning_instantiator
classifier = instantiator.last_classifier
ruleset, precisions = instantiator.last_rules

print(f"SkopeRules extracted {len(ruleset)} rules\n")

# Display each rule with its precision (weight)
var_names = [f"x_{i}" for i in range(1, 7)]

for i, (rule, precision) in enumerate(zip(ruleset, precisions)):
    print(f"Rule {i + 1} (precision={precision:.3f}):")
    for (feat_name, op), threshold in rule.items():
        # Map feature indices to variable names
        import re
        match = re.search(r"(\d+)$", feat_name)
        feat_idx = int(match.group(1)) if match else -1
        var_label = var_names[feat_idx] if 0 <= feat_idx < len(var_names) else feat_name
        print(f"  {var_label} {op} {threshold}")
    print()

The rules above were extracted by the SkopeRules classifier during the
most recent learning mode cycle. Each rule represents a conjunction of
conditions on decision variables, with a precision score indicating how
well the rule separates H-group from L-group solutions.

For the distance variables ($x_3$ through $x_6$), you should see
conditions constraining values near 0.5, reflecting the true Pareto-optimal
structure of DTLZ2. The shape variables ($x_1$, $x_2$) will typically
have wider bounds since they parameterize the position on the Pareto front.

## Interactive use: changing the reference point

In the original XLEMOO paper, the method is used interactively:
the decision maker provides a reference point, runs the optimization,
examines the extracted rules, and then may revise the reference point
to explore a different region of the Pareto front.

Below, we run XLEMOO twice with different reference points (via the ASF
scalarization) to show how the learning mode steers the search.

In [ ]:
import plotly.graph_objects as go

ref_points = {
    "centered": [0.5, 0.5, 0.5],
    "biased toward f_1": [0.2, 0.8, 0.8],
}

all_results = {}

for label, rp in ref_points.items():
    problem = dtlz2(n_objectives=3, n_variables=6)
    opts = xlemoo_options()
    opts.template.termination.max_generations = 100
    opts.template.darwin_iterations_per_cycle = 19
    opts.template.learning_iterations_per_cycle = 1
    opts.template.scalarization = ScalarizationSpec(
        type="asf",
        symbol="scal_fitness",
        reference_point=rp,
    )
    opts.template.learning_instantiator = XLEMOOSelectorOptions(
        ml_model_type="SkopeRules",
        ml_model_kwargs={"precision_min": 0.1, "n_estimators": 30},
        h_split=0.2,
        l_split=0.2,
        instantiation_factor=10.0,
        unique_only=True,
    )

    solver, extras = emo_constructor(emo_options=opts, problem=problem)
    all_results[label] = solver()

# Display objective ranges for each reference point
for label, res in all_results.items():
    obj = res.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
    print(f"Reference point: {ref_points[label]} ({label})")
    print(f"  Objective ranges: {obj.min(axis=0).round(3)} to {obj.max(axis=0).round(3)}")
    print()

In [ ]:
fig = go.Figure()
colors = ["blue", "red"]

for (label, res), color in zip(all_results.items(), colors):
    obj = res.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
    fig.add_trace(
        go.Scatter3d(
            x=obj[:, 0], y=obj[:, 1], z=obj[:, 2],
            mode="markers",
            marker=dict(size=3, color=color, opacity=0.6),
            name=label,
        )
    )

fig.update_layout(
    title="XLEMOO with different reference points on DTLZ2",
    scene=dict(xaxis_title="f_1", yaxis_title="f_2", zaxis_title="f_3"),
)
fig.show(renderer="notebook", include_plotlyjs="cdn")

The biased reference point `[0.2, 0.8, 0.8]` concentrates the H-group on
solutions with low $f_1$, so the ML model learns rules that characterize
that region. This demonstrates how XLEMOO can be steered interactively.

## Configuration reference

The scalarization function is configured via `ScalarizationSpec` on the template,
while the ML learning mode is configured via `XLEMOOSelectorOptions`
(`options.template.learning_instantiator`).

### ScalarizationSpec (options.template.scalarization)

| Parameter | Default | Description |
|---|---|---|
| `type` | `"weighted_sums"` | Scalarization type: `"weighted_sums"` or `"asf"` |
| `symbol` | `"scal_fitness"` | Column name for the scalarization value |
| `weights` | `None` (equal) | Weights per objective for weighted_sums |
| `reference_point` | `None` | Reference point for ASF (required when type="asf") |
| `delta` | `0.000001` | Delta parameter for ASF |
| `rho` | `0.000001` | Rho parameter for ASF |

### XLEMOOSelectorOptions (options.template.learning_instantiator)

| Parameter | Paper default | Description |
|---|---|---|
| `darwin_iterations_per_cycle` | 19 | EA generations per cycle before switching to learning mode |
| `learning_iterations_per_cycle` | 1 | Learning mode iterations per cycle |
| `ml_model_type` | `SkopeRules` | Interpretable ML classifier to use |
| `h_split` | 0.2 | Fraction of best solutions for the H-group |
| `l_split` | 0.2 | Fraction of worst solutions for the L-group |
| `instantiation_factor` | 10.0 | How many candidates to generate (multiplier of population size) |
| `generation_lookback` | 0 | How many recent generations to use (0 = all history) |
| `ancestral_recall` | 0 | Earliest generations to always include |
| `unique_only` | True | Deduplicate solutions before training |

### Supported ML models

- `SkopeRules` (from [imodels](https://github.com/csinva/imodels)) — default, generates interpretable rule sets
- `DecisionTree` (scikit-learn `DecisionTreeClassifier`) — fast, easily visualized
- `Slipper` (from imodels) — boosted rule learning
- `BoostedRules` (from imodels) — boosted rule ensembles

## Customizing the ML model

You can change the ML model and pass additional keyword arguments to its
constructor. Below we use a `DecisionTree` with fewer generations.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_options()
options.template.termination.max_generations = 50
options.template.darwin_iterations_per_cycle = 19
options.template.learning_iterations_per_cycle = 1
options.template.scalarization = ScalarizationSpec(
    type="asf",
    symbol="scal_fitness",
    reference_point=[0.5, 0.5, 0.5],
)

# Use DecisionTree instead of SkopeRules
options.template.learning_instantiator = XLEMOOSelectorOptions(
    ml_model_type="DecisionTree",
    ml_model_kwargs={"max_depth": 4},
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=10.0,
    unique_only=True,
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")

## Generation lookback and ancestral recall

By default, the learning mode uses the entire population history to train
the ML model (`generation_lookback=0`). You can limit this to only recent
generations, which may be useful when you want the ML model to focus on
recently discovered good solutions.

- `generation_lookback`: Only use the last N generations (0 = use all).
- `ancestral_recall`: Always include the first N generations alongside
  the lookback window, preserving early exploration diversity.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_options()
options.template.termination.max_generations = 60
options.template.darwin_iterations_per_cycle = 19
options.template.learning_iterations_per_cycle = 1
options.template.scalarization = ScalarizationSpec(
    type="asf",
    symbol="scal_fitness",
    reference_point=[0.5, 0.5, 0.5],
)

# Only look at the last 20 generations, but always include the first 3
options.template.learning_instantiator = XLEMOOSelectorOptions(
    ml_model_type="SkopeRules",
    ml_model_kwargs={"precision_min": 0.1, "n_estimators": 30},
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=10.0,
    unique_only=True,
    generation_lookback=20,
    ancestral_recall=3,
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

print(f"Solutions found: {len(results.optimal_outputs)}")

## Comparing XLEMOO with standard IBEA

A natural comparison is against standard IBEA on the same problem and budget.
XLEMOO uses a `ScalarizationSelector` with ASF, while IBEA uses its native
indicator-based selection.

In [ ]:
from desdeo.emo.options.algorithms import ibea_options, xlemoo_options

problem = dtlz2(n_objectives=3, n_variables=6)
max_gens = 100

# Standard IBEA
ibea_opts = ibea_options()
ibea_opts.template.termination.max_generations = max_gens
ibea_solver, _ = emo_constructor(emo_options=ibea_opts, problem=problem)
ibea_results = ibea_solver()

# XLEMOO with ASF
xlemoo_opts = xlemoo_options()
xlemoo_opts.template.termination.max_generations = max_gens
xlemoo_opts.template.darwin_iterations_per_cycle = 19
xlemoo_opts.template.learning_iterations_per_cycle = 1
xlemoo_opts.template.scalarization = ScalarizationSpec(
    type="asf",
    symbol="scal_fitness",
    reference_point=[0.5, 0.5, 0.5],
)
xlemoo_opts.template.learning_instantiator = XLEMOOSelectorOptions(
    ml_model_type="SkopeRules",
    ml_model_kwargs={"precision_min": 0.1, "n_estimators": 30},
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=10.0,
    unique_only=True,
)
xlemoo_solver, _ = emo_constructor(emo_options=xlemoo_opts, problem=problem)
xlemoo_results = xlemoo_solver()

# Compare distance to unit sphere
ibea_obj = ibea_results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()
xlemoo_obj = xlemoo_results.optimal_outputs[["f_1", "f_2", "f_3"]].to_numpy()

ibea_norms = np.sqrt(np.sum(ibea_obj**2, axis=1))
xlemoo_norms = np.sqrt(np.sum(xlemoo_obj**2, axis=1))

print(f"{'':>20} {'IBEA':>10} {'XLEMOO':>14}")
print(f"{'Solutions found':>20} {len(ibea_obj):>10} {len(xlemoo_obj):>14}")
print(f"{'Median norm':>20} {np.median(ibea_norms):>10.4f} {np.median(xlemoo_norms):>14.4f}")
print(f"{'Mean norm':>20} {np.mean(ibea_norms):>10.4f} {np.mean(xlemoo_norms):>14.4f}")

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=ibea_obj[:, 0], y=ibea_obj[:, 1], z=ibea_obj[:, 2],
        mode="markers",
        marker=dict(size=3, color="blue", opacity=0.6),
        name="IBEA",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=xlemoo_obj[:, 0], y=xlemoo_obj[:, 1], z=xlemoo_obj[:, 2],
        mode="markers",
        marker=dict(size=3, color="red", opacity=0.6),
        name="XLEMOO-IBEA",
    )
)

fig.update_layout(
    title="IBEA vs XLEMOO-IBEA on DTLZ2",
    scene=dict(xaxis_title="f_1", yaxis_title="f_2", zaxis_title="f_3"),
)
fig.show(renderer="notebook", include_plotlyjs="cdn")

## Accessing the learning archive

The `ConstructorExtras` object returned by `emo_constructor` includes a
`learning_archive` that stores the full history of all evaluated solutions
(from both Darwinian and Learning modes). This can be used for post-hoc
analysis.

In [ ]:
problem = dtlz2(n_objectives=3, n_variables=6)

options = xlemoo_options()
options.template.termination.max_generations = 50
options.template.darwin_iterations_per_cycle = 19
options.template.learning_iterations_per_cycle = 1
options.template.scalarization = ScalarizationSpec(
    type="asf",
    symbol="scal_fitness",
    reference_point=[0.5, 0.5, 0.5],
)
options.template.learning_instantiator = XLEMOOSelectorOptions(
    ml_model_type="SkopeRules",
    ml_model_kwargs={"precision_min": 0.1, "n_estimators": 30},
    h_split=0.2,
    l_split=0.2,
    instantiation_factor=10.0,
    unique_only=True,
)

solver, extras = emo_constructor(emo_options=options, problem=problem)
results = solver()

# The learning archive contains all evaluated solutions with generation numbers
archive_df = extras.learning_archive.solutions
print(f"Total solutions in learning archive: {len(archive_df)}")
print(f"Generations covered: {archive_df['generation'].min()} to {archive_df['generation'].max()}")
print(f"\nArchive columns: {archive_df.columns}")

## Summary

XLEMOO extends standard evolutionary algorithms with interpretable machine
learning to provide both optimization and explainability. Key takeaways:

- Use `xlemoo_options()` for a ready-to-run configuration with a
  `ScalarizationSelector` and weighted sums scalarization.
- Configure the `ScalarizationSpec` on the template to choose between
  `"weighted_sums"` and `"asf"` scalarization. Both the Darwinian selector
  and the learning mode instantiator use the same scalarization column.
- The **ASF** with a decision-maker-supplied reference point is the
  recommended configuration (as in the original paper), guiding the learning
  mode toward the preferred region of the Pareto front.
- **SkopeRulesClassifier** is the recommended ML model for interpretable
  rule extraction.
- Use the `learning_archive` and rule extraction utilities for post-hoc
  analysis and explainability.
- XLEMOO supports interactive use: examine the extracted rules, then re-run
  with a modified reference point to explore other regions.